*Requested third-party library：numpy, pyedflib*

In [1]:
import json

with open('raw_clean_mapping.json', 'r') as f:
    mapping_dict = json.load(f)

In [ ]:
import json
with open("config.json", "rt") as f:
    config_obj = json.load(f)
print(config_obj)
globals().update(config_obj)

{'dataset_path': '<path to your chb-mit directory> e.g ./physionet.org/files/chbmit/1.0.0/', 'clean_data_path': '<path to store clean & aligned dataset> e.g ./data_clean', 'label_output_path': '<path to store generated TOML files> e.g ./ref_labels', 'ignore_lst': ['chb16_18.edf', 'chb16_19.edf', 'chb17c_13.edf', 'chb18_01.edf', 'chb19_01.edf', 'chb11_01.edf', 'chb12_27.edf', 'chb12_28.edf', 'chb12_29.edf', 'chb09_01.edf', 'chb15_01.edf']}


In [2]:
# 萃取公共通道信息并顺便完成数据类型转换 Extract common channel data and convert data types by the way
import os
import os.path
from pyedflib.highlevel import read_edf
import numpy as np

outdir = clean_data_path
for k, v in mapping_dict.items():
    patdir = os.path.join(outdir, k[-5:])
    os.makedirs(patdir, mode=0o755, exist_ok=True)
    chs, files = v[0], v[1]
    for fn, seq_lst in files:
        content, _, _ = read_edf(os.path.join(k, fn), digital=False)
        # new_data = np.empty((len(chs), content.shape[1]))
        new_data = content.astype(dtype=np.float32)[np.array([seq_lst])]
        np.save(os.path.join(patdir, fn[:-4]), new_data.squeeze()) # TODO Test Needed Here
        print(fn, end=' ')

chb01_01.edf chb01_02.edf chb01_03.edf chb01_04.edf chb01_05.edf chb01_06.edf chb01_07.edf chb01_08.edf chb01_09.edf chb01_10.edf chb01_11.edf chb01_12.edf chb01_13.edf chb01_14.edf chb01_15.edf chb01_16.edf chb01_17.edf chb01_18.edf chb01_19.edf chb01_20.edf chb01_21.edf chb01_22.edf chb01_23.edf chb01_24.edf chb01_25.edf chb01_26.edf chb01_27.edf chb01_29.edf chb01_30.edf chb01_31.edf chb01_32.edf chb01_33.edf chb01_34.edf chb01_36.edf chb01_37.edf chb01_38.edf chb01_39.edf chb01_40.edf chb01_41.edf chb01_42.edf chb01_43.edf chb01_46.edf chb02_01.edf chb02_02.edf chb02_03.edf chb02_04.edf chb02_05.edf chb02_06.edf chb02_07.edf chb02_08.edf chb02_09.edf chb02_10.edf chb02_11.edf chb02_12.edf chb02_13.edf chb02_14.edf chb02_15.edf chb02_16+.edf chb02_16.edf chb02_17.edf chb02_18.edf chb02_19.edf chb02_20.edf chb02_21.edf chb02_22.edf chb02_23.edf chb02_24.edf chb02_25.edf chb02_26.edf chb02_27.edf chb02_28.edf chb02_29.edf chb02_30.edf chb02_31.edf chb02_32.edf chb02_33.edf chb02_34.ed

In [4]:
# Generate channel_info jsons recording shuffling info
import os.path
import numpy as np

outdir = clean_data_path
for k, v in mapping_dict.items():
    patdir = os.path.join(outdir, k[-5:])
    os.makedirs(patdir, mode=0o755, exist_ok=True)
    chs, files = v[0], v[1]
    pat_json = {'Channels': chs, 'EDF Files': []}
    for fn, seq_lst in files:
        pat_json['EDF Files'].append({'File Name': fn, 'Original Index': seq_lst})
    with open(os.path.join(patdir, 'channel_info.json'), 'w') as f:
        json.dump(pat_json, f, indent=2)
    

In [ ]:
from pyedflib import EdfReader

class EdfReaderWrapper(EdfReader):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def __enter__(self):
        return self
    def __exit__(self, *args):
        super().close()

In [9]:
# From chb{02d}-summary.txt generate datetime_info jsons
import os
import json
import datetime
import pyedflib

outdir = clean_data_path
data_dir = dataset_path
# ignore_lst

# TODO Combined with command-line arguments
with open(os.path.join("..", "global_config.json"), "rt") as f:
    config_obj = json.load(f)
# globals().update(config_obj)
dt_fmt = config_obj.datetime_fmt

for indexPatient in range(1, 24+1):
    pat_json = []
    with open(os.path.join(data_dir, f"chb{indexPatient:02d}", f"chb{indexPatient:02d}-summary.txt"), 'r') as f:
        for line in f:
            data=line.split(':')
            if(data[0]=="File Name"):
                edfName=data[1].strip()
                if edfName in ignore_lst:
                    continue
                with EdfReaderWrapper(os.path.join(data_dir, f"chb{indexPatient:02d}", edfName)) as pedf: 
                    startTime = pedf.getStartdatetime()
                    startStr = startTime.strftime(dt_fmt)
                    endTime = startTime + datetime.timedelta(seconds=pedf.getFileDuration())
                    endStr = endTime.strftime(dt_fmt)

                    pat_json.append({'File Name': edfName[:-3]+'npy', 'Record Datetimes': [startStr, endStr], 'Seizures': []}) # TODO Add Annotations Retrieval
                    
                    nextLine = f.readline().strip()
                    while (len(nextLine) != 0 and not nextLine.startswith('Number of Seizures in File:')):
                        nextLine = f.readline().strip()
                    if nextLine.startswith('Number of Seizures in File:'):
                        for j in range(0, int(nextLine.split(':')[1])):
                            szStartSec = int(f.readline().split(': ')[1].strip().split(' ')[0])
                            szEndSec = int(f.readline().split(': ')[1].strip().split(' ')[0])
                            pat_json[-1]['Seizures'].append([szStartSec, szEndSec]))

        
    with open(os.path.join(outdir, f'chb{indexPatient:02d}', 'datetime_info.json'), 'w') as f:
        json.dump(pat_json, f, indent=2)